In [1]:
import logging
import psycopg2 as pg

# - *dbname*: the database name
# - *database*: the database name (only as keyword argument)
# - *user*: user name used to authenticate
# - *password*: password used to authenticate
# - *host*: database host address (defaults to UNIX socket if not provided)
# - *port*: connection port number (defaults to 5432 if not provided)
connection = pg.connect(
    user='Tesla', database='postgres', host='localhost', port=6543
)
cursor = connection.cursor()

# connection.close()

In [3]:
# Display all Tables Present in Database
def show_tables():
    try:
        Query = """
    select tablename from pg_tables where tablename not like 'pg_%';
    """
        cursor.execute(Query)
        for i, x in enumerate(cursor.fetchall(), start=1):
            print(f'{i:^5d} : {x[0]}')
        connection.commit()
    except Exception as e:
        logging.exception(e)
        connection.rollback()

In [4]:
## Arrays
try:
    # select '3rd' element from Array
    Query = """
WITH arr AS (SELECT ARRAY[0,1,2,3,4,5,6,7,8,9,0] int_arr) SELECT int_arr[3] FROM arr;
"""
    cursor.execute(Query)
    print(cursor.description)
    for i, x in enumerate(cursor.fetchmany(100), start=1):
        print(f'{i:^5d} : {x[0]}')

    # select '2nd row 3rd Column element'
    Query = """
WITH arr AS (SELECT ARRAY[[0,1,2], [3,4,5], [6,7,8], [4,3,65]] int_arr) SELECT int_arr[2][3] FROM arr;
"""
    cursor.execute(Query)
    print(cursor.description)
    for i, x in enumerate(cursor.fetchmany(100), start=1):
        print(f'{i:^5d} : {x[0]}')

    # select elements by index Range from Array '3rd to 7th element'
    Query = """
WITH arr AS (SELECT ARRAY[0,1,2,3,4,5,6,7,8,9,0] int_arr) SELECT int_arr[3:7] FROM arr;
"""
    cursor.execute(Query)
    print(cursor.description)
    for i, x in enumerate(cursor.fetchmany(100), start=1):
        print(f'{i:^5d} : {x[0]}')
    # array dimensions 1 row 3 column
    Query = """
WITH arr AS (SELECT ARRAY[0,1,2] int_arr) SELECT ARRAY_DIMS(int_arr) FROM arr;
"""
    cursor.execute(Query)
    print(cursor.description)
    for i, x in enumerate(cursor.fetchmany(100), start=1):
        print(f'{i:^5d} : {x[0]}')
    # array dimensions 4 row 3 column
    Query = """
WITH arr AS (SELECT ARRAY[[0,1,2], [3,4,5], [6,7,8], [4,3,65]] int_arr) SELECT ARRAY_DIMS(int_arr) FROM arr;
"""
    cursor.execute(Query)
    print(cursor.description)
    for i, x in enumerate(cursor.fetchmany(100), start=1):
        print(f'{i:^5d} : {x[0]}')

    connection.commit()
except Exception as e:
    logging.exception(e)
    connection.rollback()

(Column(name='int_arr', type_code=23),)
  1   : 2
(Column(name='int_arr', type_code=23),)
  1   : 5
(Column(name='int_arr', type_code=1007),)
  1   : [2, 3, 4, 5, 6]
(Column(name='array_dims', type_code=25),)
  1   : [1:3]
(Column(name='array_dims', type_code=25),)
  1   : [1:4][1:3]


In [5]:
# Datetime Operations
try:
    Query = """
SELECT (DATE_TRUNC('MONTH', ('202202'||'01')::DATE) + INTERVAL '1 MONTH - 1 day')::DATE;
"""
    cursor.execute(Query)
    print(cursor.description)
    for i, x in enumerate(cursor.fetchmany(100), start=1):
        print(f'{i:^5d} : {x[0]}')

    Query = """
SELECT TO_CHAR('2022-01-16 16:40:32'::TIMESTAMP, 'DD Mon YYYY HH:MI:SSPM');
"""
    cursor.execute(Query)
    print(cursor.description)
    for i, x in enumerate(cursor.fetchmany(100), start=1):
        print(f'{i:^5d} : {x[0]}')

    Query = """
SELECT TO_CHAR('2023-01-16 16:40:32'::TIMESTAMP,
 '"Today is "FMDay", the "DDth" day of the month of "FMMonth" of "YYYY');
"""
    cursor.execute(Query)
    print(cursor.description)
    for i, x in enumerate(cursor.fetchmany(100), start=1):
        print(f'{i:^5d} : {x[0]}')

    Query = """
SELECT TO_CHAR('2016-08-12 16:40:32'::TIMESTAMP, 'TMDay, DD" de "TMMonth" del año "YYYY')
"""
    cursor.execute(Query)
    print(cursor.description)
    for i, x in enumerate(cursor.fetchmany(100), start=1):
        print(f'{i:^5d} : {x[0]}')
    connection.commit()
except Exception as e:
    logging.exception(e)
    connection.rollback()

(Column(name='date', type_code=1082),)
  1   : 2022-02-28
(Column(name='to_char', type_code=25),)
  1   : 16 Jan 2022 04:40:32PM
(Column(name='to_char', type_code=25),)
  1   : Today is Monday, the 16th day of the month of January of 2023
(Column(name='to_char', type_code=25),)
  1   : Friday, 12 de August del año 2016


In [6]:
try:
    Query = """
CREATE TABLE if not exists person (
 person_id BIGINT NOT NULL,
 last_name VARCHAR(255) NOT NULL,
 first_name VARCHAR(255),
 age INT NOT NULL,
 PRIMARY KEY (person_id)
);
"""
    cursor.execute(Query)
    show_tables()
    connection.commit()
except Exception as e:
    logging.exception(e)
    connection.rollback()

  1   : person_pk
  2   : person
  3   : sql_features
  4   : sql_implementation_info
  5   : sql_parts
  6   : sql_sizing


In [7]:
try:
    Query = """
CREATE UNLOGGED TABLE person_unlogged (
 person_id BIGINT NOT NULL PRIMARY KEY,
 last_name VARCHAR(255) NOT NULL,
 first_name VARCHAR(255),
 address VARCHAR(255),
 city VARCHAR(255)
);
"""
    cursor.execute(Query)
    connection.commit()
    show_tables()

except Exception as e:
    logging.exception(e)
    connection.rollback()

  1   : person_pk
  2   : person
  3   : person_unlogged
  4   : sql_features
  5   : sql_implementation_info
  6   : sql_parts
  7   : sql_sizing


In [9]:
try:
    Query = """
CREATE TABLE if not exists person_pk (
 person_id BIGINT NOT NULL,
 last_name VARCHAR(255) NOT NULL,
 first_name VARCHAR(255),
 address VARCHAR(255),
 city VARCHAR(255),
 PRIMARY KEY (person_id)
);
"""
    cursor.execute(Query)
    connection.commit()
    show_tables()

except Exception as e:
    logging.exception(e)
    connection.rollback()

  1   : person_pk
  2   : person
  3   : person_unlogged
  4   : sql_features
  5   : sql_implementation_info
  6   : sql_parts
  7   : sql_sizing


In [ ]:
try:
    Query = """

"""
    cursor.execute(Query)
    print(cursor.description)
    for i, x in enumerate(cursor.fetchmany(100), start=1):
        print(f'{i:^5d} : {x[0]}')

    connection.commit()
except Exception as e:
    logging.exception(e)
    connection.rollback()